In [ ]:
import spacy
import en_core_web_sm
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

from sklearn.metrics.pairwise import cosine_similarity
import random

In [ ]:
users_input = {'text' :'',
               'image' : ''}

output_to_engine = {'engine': '',
                    'request':
                     {'text':'',
                      'image':''}
                    }
engine_output = {'text': '', 'link': ''} # ... waiting for engines output formats to be decided

output_to_user = {'answer': ''}

# Engine Selection - Step 1
# (format - based)

In [ ]:
def user_input_format(user_input):
  if user_input['image']!='':
    if user_input['text']=='':
      return 'image'
    else:
      return 'text + image'
  if user_input['text']!='':
    return 'text'

# Engine Selection 2a

In [ ]:
SbI_list = ['Show me images similar to this one.', 'Find me images that look like this one.']
vQA_list = ['How many vessels does this image show?', 'Is this a rural or an urban area?']

SbI_embeddings = model.encode(SbI_list)
vQA_embeddings = model.encode(vQA_list)

In [ ]:
def engine_selection_2a(text):
  text_embeddings = [model.encode(text)]

  if cosine_similarity(text_embeddings,SbI_embeddings[:]).max() > cosine_similarity(text_embeddings,vQA_embeddings[:]).max():
    return 'SbI'
  else:
    return 'vQA'

# Engine Selection 2b

In [ ]:
def request_disambiguation(text):
  disambiguation = {'need' : False,
                    'message' : '' }
  if ' near ' in users_input['text']:
    disambiguation['need'] = True
    disambiguation['message'] = "Can you repeat your question replacing 'near' with a specific distance, please?"

  return disambiguation

In [ ]:
def existence_of_geographical_object(textual_input):
  geo_object_presense = False
  nlp = en_core_web_sm.load()
  doc = nlp(textual_input)

  for X in doc.ents:
    if X.label_ in ['GPE','FAC','LOC']:
      geo_object_presense = True
      break

  return geo_object_presense

In [ ]:
def engine_selection_2b(text):
  if existence_of_geographical_object(text) == True:
    return 'EarthQA'
  else:
    return 'SbT'

## choose between Chat and EarthQA enamples

In [ ]:
engine_selection_2b('Find me images with vessels near Genoa port.') # !important! geographical objects have to be in English.. eg Genoa , not Genova

'EarthQA'

In [ ]:
engine_selection_2b('Find me images with vessels in Aliakmonas')

'EarthQA'

In [ ]:
engine_selection_2b('Find me images with mountains near zagorohoria')

'EarthQA'

In [ ]:
engine_selection_2b('Find me images with mountains near Itea.')

'EarthQA'

In [ ]:
engine_selection_2b('Find me images with mountains in london.')

'EarthQA'

In [ ]:
engine_selection_2b('Find me images with mountains.')

'SbT'

## chat/textual engine decision

In [ ]:
chat_list = ['Thank you!', 'This was all I wanted']
engine_list = ['Show me images containing vessels', 'Find me Sentinel-2 satellite images that show Mount Etna, have been taken in February 2021 and have cloud cover less than 10%).']

In [ ]:
chat_embeddings = model.encode(chat_list)
engine_embeddings = model.encode(engine_list)

In [ ]:
def request_to_textual_engine(text):
  # when we have a dataset of user's requests, we can implement this function via a binary classifier
  request = False
  # find users request's embedding
  text_embeddings = [model.encode(text)]

  if cosine_similarity(text_embeddings,chat_embeddings[:]).max() < cosine_similarity(text_embeddings,engine_embeddings[:]).max():
    request = True

  return request

# Response enhancement

In [ ]:
def response_enhancement(engine, answer):
#... waiting for engines output formats to be decided
  assist_list = ['What else can I help you with?', 'I will be glad to help you with your next request. :)', 'What else can I assist you with?', 'Pose another request, please.']
  response = engine +  ' answer.\n' + random.choice(assist_list)
  return response

# crucial function

In [ ]:
def digital_assistant_to_engine(users_input):

  #Engine Selection step 1
  # path 'text with image'
  if  user_input_format(users_input) == 'text + image':
    # Engine Selection 2a
    output_to_engine['engine'] = engine_selection_2a(users_input['text'])
    output_to_engine['request'] = users_input

  #path 'textual'
  elif user_input_format(users_input) == 'text':
    # decide between chat and textual engine
    if request_to_textual_engine(users_input['text']) == False :
      output_to_engine['engine'] = 'conversational'
      output_to_engine['request'] = users_input
    else:
      # asking for clarifications
      if request_disambiguation(users_input['text'])['need'] == True:
        return request_disambiguation(users_input['text'])['message']

      # Engine Selection 2b
      output_to_engine['engine'] = engine_selection_2b(users_input['text'])
      output_to_engine['request'] = users_input

  return output_to_engine

# main ()

In [ ]:
def main():
    messages = [{"role": "assistant", "content": "Welcome to DA4DTE! Please enter your request.",
                         'image': ''}]
    while True:
        # read users_input file
        messages.append({"role": "user", "content": users_input['text'],
                         'image': users_input['image']})

        if type(digital_assistant_to_engine(users_input))== str: # meaning that disambiguation is needed
          answer = digital_assistant_to_engine(users_input)

        else:
          engine_input = digital_assistant_to_engine(users_input)
          # the engine_input json file is available
          #[... waiting for the engine to respond ...]
          # TI reads engine_output json file
          answer = response_enhancement(engine_output)
          # the output_to_user file is available

        messages.append({"role": "assistant", "content": answer, "image":''})

# demo

In [ ]:
def demo():
    messages = [{"role": "assistant", "content": "Welcome to DA4DTE! Please enter your request.",
                         'image': ''}]
    while True:
        # read users_input file
        file = input("INSERT FILE NUMBER:  1-5. Type 'exit' to exit.")

        if file=='exit':
          for m in messages:
            print(m)
          break

        with open("/content/gdrive/MyDrive/Colab Notebooks/DA4DTE TI/example"+file+".json", "r") as file:
          users_input = json.load(file)

        print('\nUser:')
        print(users_input)

        messages.append({"role": "user", "content": users_input['text'],
                         'image': users_input['image']})

        if type(digital_assistant_to_engine(users_input))== str: # meaning that disambiguation is needed
          answer = digital_assistant_to_engine(users_input)

        else:
          engine_input = digital_assistant_to_engine(users_input)
          # the engine_input json file is available
          #[... waiting for the engine to respond ...]
          engine_output = engine_input['engine'] ## temporarily
          # TI reads engine_output json file
          answer = response_enhancement(engine_output,'')
          # the output_to_user file is available

          print('DA4DTE:')
          print(answer+'\n')

        messages.append({"role": "assistant", "content": answer, "image":''})



In [ ]:
# create some user's inputs json files
import json
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
demo()

INSERT FILE NUMBER:  1-5. Type 'exit' to exit.1

User:
{'text': 'Give me an image with vessels.', 'image': ''}
DA4DTE:
SbT answer.
What else can I assist you with?

INSERT FILE NUMBER:  1-5. Type 'exit' to exit.3

User:
{'text': 'Show me 100 similar images.', 'image': 'IMAGE'}
DA4DTE:
SbI answer.
What else can I assist you with?

INSERT FILE NUMBER:  1-5. Type 'exit' to exit.2

User:
{'text': 'Retrieve Sentinel-2 images containing marine litters in the Mediterranean sea.', 'image': ''}
DA4DTE:
EarthQA answer.
What else can I help you with?

INSERT FILE NUMBER:  1-5. Type 'exit' to exit.4

User:
{'text': 'Is this a rural or an urban area?', 'image': 'IMAGE'}
DA4DTE:
vQA answer.
What else can I help you with?

INSERT FILE NUMBER:  1-5. Type 'exit' to exit.5

User:
{'text': 'Thank you', 'image': ''}
DA4DTE:
conversational answer.
What else can I help you with?

INSERT FILE NUMBER:  1-5. Type 'exit' to exit.exit
{'role': 'assistant', 'content': 'Welcome to DA4DTE! Please enter your request

# examples

In [ ]:
users_input_x = {'text' :'Give me an image with vessels.',
               'image' : ''}
# Serializing json
json_object = json.dumps(users_input_x, indent=4)

# Writing to user_request_x.json
with open("/content/gdrive/MyDrive/Colab Notebooks/DA4DTE TI/example1.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
users_input_x = {'text' :'Retrieve Sentinel-2 images containing marine litters in the Mediterranean sea.',
               'image' : ''}
# Serializing json
json_object = json.dumps(users_input_x, indent=4)

# Writing to user_request_x.json
with open("/content/gdrive/MyDrive/Colab Notebooks/DA4DTE TI/example2.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
users_input_x = {'text' :'Show me 100 similar images.',
               'image' : 'IMAGE'}
# Serializing json
json_object = json.dumps(users_input_x, indent=4)

# Writing to user_request_x.json
with open("/content/gdrive/MyDrive/Colab Notebooks/DA4DTE TI/example3.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
users_input_x = {'text' :'Is this a rural or an urban area?',
               'image' : 'IMAGE'}
# Serializing json
json_object = json.dumps(users_input_x, indent=4)

# Writing to user_request_x.json
with open("/content/gdrive/MyDrive/Colab Notebooks/DA4DTE TI/example4.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
users_input_x = {'text' :'Thank you',
               'image' : ''}
# Serializing json
json_object = json.dumps(users_input_x, indent=4)

# Writing to user_request_x.json
with open("/content/gdrive/MyDrive/Colab Notebooks/DA4DTE TI/example5.json", "w") as outfile:
    outfile.write(json_object)